In [ ]:
from numpy import pi as π
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import tqdm
import firedrake
from firedrake import inner, grad, dx, ds, dS, max_value, Constant
import irksome
from irksome import Dt

In [ ]:
num_levels = 5
mesh = firedrake.UnitDiskMesh(num_levels)
bernstein = firedrake.FiniteElement("Bernstein", "triangle", 2)
bernstein_dg = firedrake.BrokenElement(bernstein)
Q = firedrake.FunctionSpace(mesh, bernstein_dg)

In [ ]:
x = firedrake.SpatialCoordinate(mesh)
u = firedrake.as_vector((-x[1], x[0]))

Create the initial data -- a spherical blip of radius 1/3 centered at the point (0.5, 0.0).

In [ ]:
ξ = Constant((0.5, 0.0))
r = Constant(1 / 3)
expr = max_value(0, 1 - inner(x - ξ, x - ξ) / r**2)
ϕ_ = firedrake.project(expr, Q)

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.tripcolor(ϕ_, axes=ax)
fig.colorbar(colors);

The sink term will be a spherical blob of radius 1/6 centered at the point (-0.5, 0).

In [ ]:
a_0 = Constant(10.0)
ζ = Constant((-0.5, 0.0))
ρ = Constant(1 / 6)
expr = -a_0 * max_value(0, 1 - inner(x - ζ, x - ζ) / ρ**2)
a = firedrake.project(expr, Q)

Create the variational form of the advection equation.

In [ ]:
ψ = firedrake.TestFunction(Q)
F_1 = (Dt(ϕ_) * ψ - inner(ϕ_ * u, grad(ψ)) - a * ψ) * dx
ν = firedrake.FacetNormal(mesh)
u_n = firedrake.max_value(0, inner(u, ν))
F_2 = (ϕ_("+") * u_n("+") - ϕ_("-") * u_n("-")) * (ψ("+") - ψ("-")) * dS
F = F_1 + F_2

t = Constant(0.0)
num_steps = 512
dt = Constant(2 * π / num_steps)

Get the form and stages for the Lobatto-IIIC time discretization of this problem from Irksome.
Next get the test functions used in that form.
We'll need to add an extra term to the form that includes the next solution value explicitly, and that will mean replacing the stage variables and test functions using the `firedrake.replace` function.

In [ ]:
tableau = irksome.LobattoIIIC(2)
old_form, old_stages, bcs, nullspaces, bcdata = irksome.getForm(F, tableau, t, dt, ϕ_)
old_test_fns = firedrake.split(old_form.arguments()[0])

**Heads up: this is the important part.**
So the value of the solution at the next timestep is
$$\phi_{n + 1} = \phi_n + \delta t\cdot \sum_i\beta_ik_i$$
where $\{k_i\}$ are the stages of the RK method.
We want to enforce the inequality constraint that $\phi_{n + 1} \ge 0$, keeping in mind that we're solving for the stages $\{k_i\}$.
Mathematically, there's nothing wrong with this inequality constraint as such.
PETSc, however, only allows us to do *box* constraints -- inequality constraints of the form $u_1 \le u \le u_2$.
Ours is of the more general form $u_1 \le Au + f \le u_2$, i.e. there's a linear transformation in there albeit a pretty trivial one.

The way out of the dilemma is that we need to add $\phi_{n + 1}$ explicitly as an unknown to the problem and solve simultaneously for it along with the RK stages.
We'll enforce both the equality constraint that $\phi_{n + 1} = \phi_n + \delta t\cdot \sum_i\beta_ik_i$ along with the inequality constraint $\phi_{n + 1} \ge 0$, which is now a simple box constraint.

To make the fix, what we'll do is first create a new function space $Z$ where we stack an extra copy $Q$ of the solution space where $\phi_{n + 1}$ lives onto the (mixed) function space $S$ where all the stages live.

In [ ]:
S = old_stages.function_space()
Z = Q * S

Then we get a new function from the expanded solution $\times$ stages space $Z$ and a set of test functions.
Split them into the solution and stages parts.

In [ ]:
zs = firedrake.Function(Z)
soln_stages = firedrake.split(zs)
ws = firedrake.TestFunctions(Z)

ϕ, new_stages = soln_stages[0], soln_stages[1:]
ψ, new_test_fns = ws[0], ws[1:]

Ok and here's the real crux of the problem.
Create some dictionaries to help us map the old solution and stages into the new joint solution $\times$ stage variables.

In [ ]:
stage_dict = {k_old: k_new for k_old, k_new in zip(old_stages, new_stages)}
test_fn_dict = {q_old: q_new for q_old, q_new in zip(old_test_fns, new_test_fns)}

Create a new part of the form (which we'll call `soln_form`) to represent the constraint that the next value of the solution is
$$\phi_{n + 1} = \phi_n + \delta t\sum_s\beta_sk_s$$
and add it to the old form.

In [ ]:
soln_form = (ϕ - (ϕ_ + dt * sum(β * k for β, k in zip(tableau.b, new_stages)))) * ψ * dx
form = soln_form + firedrake.replace(old_form, {**stage_dict, **test_fn_dict})

Create some functions to hold the upper and lower bounds for the solution.
All of them have an upper bound of $\infty$ while the next value of the solution has a lower bound of 0.

In [ ]:
from firedrake.petsc import PETSc

upper = firedrake.Function(Z)
with upper.dat.vec as upper_vec:
    upper_vec.set(PETSc.INFINITY)

lower = firedrake.Function(Z)
with lower.dat.vec as lower_vec:
    lower_vec.set(PETSc.NINFINITY)

with lower.sub(0).dat.vec as lower_vec:
    lower_vec.set(0.0)

Create the solver and a list of variables to store the solution at every timestep.

**Aside**: PETSc has two VI solvers -- a semi-smooth Newton solver `vinewtonssls` and an active-set solver `vinewtonrsls`.
The semi-smooth Newton solver crashes pretty early on.

In [ ]:
params = {
    "mat_type": "aij",
    "snes_type": "vinewtonrsls",
    "ksp_type": "gmres",
    "pc_type": "ilu",
}
problem = firedrake.NonlinearVariationalProblem(form, zs)
solver = firedrake.NonlinearVariationalSolver(problem, solver_parameters=params)

In [ ]:
ϕs = [ϕ_.copy(deepcopy=True)]

Solve the problem forward in time.
We're using the variable `ϕ_` to store the old value of the solution; we have to explicitly assign it the newly-computed value at the end of each timestep now.

In [ ]:
for step in tqdm.trange(num_steps):
    solver.solve(bounds=(lower, upper))
    t.assign(float(t) + float(dt))
    ϕ = zs.subfunctions[0]
    ϕ_.assign(ϕ)
    ϕs.append(ϕ.copy(deepcopy=True))

Make an animation of the solution.

In [ ]:
%%capture
fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.tripcolor(
    ϕs[0], vmin=0.0, vmax=1.0, num_sample_points=4, axes=ax
)

fn_plotter = firedrake.FunctionPlotter(mesh, num_sample_points=4)
def animate(ϕ):
    colors.set_array(fn_plotter(ϕ))

interval = 1e3 * 20 / num_steps
animation = FuncAnimation(fig, animate, frames=ϕs, interval=interval)

In [ ]:
HTML(animation.to_html5_video())

Half-way through the simulation, the initial blob advects into the sink.
But by posing the problem as a variational inequality, we can make sure the solution does not go negative.
This is especially important in some physics problems where there are sinks that can remove mass from the system but where the key solution variable can't go negative.
Example 1: the solution variable represents the thickness of a glacier.
The ice can melt with sufficient solar heating, but the thickness of the glacier can't go negative.
Example 2: the solution variable represents the thickness of liquid water, e.g. from rainfall, flowing over a landscape.
This water can infiltrate into the subsurface aquifer, but the thickness of the surface water layer likewise can't go negative.

In [ ]:
volumes = [firedrake.assemble(ϕ * dx) for ϕ in ϕs]
fig, ax = plt.subplots()
ax.plot(volumes);